In [ ]:
import os
from datetime import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import csv
from csv import writer



#tensorflow import
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.optimizers import Adam

#logging
from tensorflow import keras
from keras import backend as K


#custom functions import
from ts_functions import time_window_generator
from aux_functions import aux_functions


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
#option flags
debug_mode = True
debug_output_n = 2

performance_log = True
data_log_csv_path = "training_log/tencent_t1.csv"


#tensorboard logging doesnt work for now
tf_log_dir = "logs/LSTM_stocks/"




In [ ]:
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, r"stock_data\tencent_04_22_day.csv") #stock data

In [ ]:
drop_list = ['change']

df = pd.read_csv(filename).drop(drop_list, axis=1)

In [ ]:
if debug_mode:
    display(df)

In [ ]:
if debug_mode:
    display(df.describe())

# Model DataFrame preprocessing

In [ ]:
#split data into different stock
# stock_list = [x.sort_values("trade_date", ascending = True) for _,x in df.groupby('ts_code')]

stock_list = []

for ts_code, x in df.groupby('ts_code'):
    stock_list.append(x.copy().sort_values("trade_date", ascending = True).reset_index().drop('index', axis=1))




#sort by trade_date, ascending
if debug_mode:
    print("number of stocks : ", len(stock_list))

# ADD HSI into the df

In [ ]:
hsi_filename = os.path.join(fileDir, r"stock_data\hsi_07_22_day.csv")

df_hsi = pd.read_csv(hsi_filename).drop(['index_ts_code', 'index_vol', 'index_change', 'index_vol', 'index_swing'], axis=1).sort_values("index_trade_date", ascending = True).reset_index().drop('index', axis=1)

display(df_hsi)

In [ ]:
# #append hs300 by matching dates
# invalid_stock_list = []# the stock that is fucked


# for i in range(len(stock_list)):
#     stock = stock_list[i]
#     if stock['trade_date'].iloc[-1] != df_hs300['index_trade_date'].iloc[-1]:
#         invalid_stock_list.append(i)


In [ ]:
# # for i in invalid_stock_list:
# #     print(stock_list[i].head(1))

# invalid_stock_list = sorted(invalid_stock_list, reverse=True)

# print(invalid_stock_list)


# for i in invalid_stock_list:
#     stock_list.pop(i)

In [ ]:
stock_list[0]['trade_date']
print(len(stock_list[0]['trade_date']))

tx = df_hsi[df_hsi['index_trade_date'].isin(stock_list[0]['trade_date'])].sort_values("index_trade_date", ascending = True).reset_index().drop('index', axis=1)
display(tx)

display(stock_list[0])

display(df_hsi)

txp = pd.concat([stock_list[0], tx], axis=1, join='inner')
display(txp)

In [ ]:
#concat

for i in range(len(stock_list)):

    temp = df_hsi[df_hsi['index_trade_date'].isin(stock_list[i]['trade_date'])].sort_values("index_trade_date", ascending = True).reset_index().drop('index', axis=1)
    stock_list[i] = pd.concat([stock_list[i], temp], axis=1, join='inner')#.drop('index_trade_date', axis=1)

    # stock_list[i] = stock_list[i].join(df_hsi, on='str', lsuffix='trade_date', rsuffix='index_trade_date', how='inner')





In [ ]:
display(stock_list[0])

In [ ]:
#check time matching
for stock in stock_list:
    print(stock['trade_date'].isin(stock['index_trade_date']).value_counts())

In [ ]:
display(stock_list[0][0:15])

# add time periodicity

In [ ]:
#setup timestamp for year periodicity 
date_time_db = []

for stock_i in stock_list:
    date_time_db.append(pd.to_datetime(stock_i.pop('trade_date'), format='%Y%m%d'))

In [ ]:
if debug_mode:
    print(date_time_db[0])

In [ ]:
date_time_stamp_db = []

for dt in date_time_db:
    date_time_stamp_db.append(dt.map(pd.Timestamp.timestamp))

In [ ]:
if debug_mode:
    date_time_stamp_db[0]

In [ ]:
year = 24*60*60*(365.2425)

for i in range(len(stock_list)):
    
    stock_list[i]["Year sin"] = np.sin(date_time_stamp_db[i] * (2 * np.pi / year))
    stock_list[i]["Year cos"] = np.cos(date_time_stamp_db[i] * (2 * np.pi / year))
    

In [ ]:
if debug_mode:
    plt.plot(np.array(stock_list[0]['Year sin'])[:1000])
    plt.plot(np.array(stock_list[0]['Year cos'])[:1000])
    plt.xlabel('Time [day]')
    plt.title('Time of year signal, in day')

In [ ]:
if debug_mode:
    stock_list[5].head()

In [ ]:
#df processing
for stock in stock_list:
    stock.name = stock.iloc[0,0]
    stock.pop("ts_code") #pop the stock code
    #stock.pop("pre_close") #pop previous day close price

In [ ]:
if debug_mode:
    stock_list[0].head()

# MA implementation

In [ ]:
ma_label = 'open'

In [ ]:
for stock in stock_list:
    
    #MA

    processing_row = stock[ma_label].to_frame()

    stock['sma5'] = processing_row[ma_label].rolling(5).mean()

    stock['ema5'] = processing_row[ma_label].ewm(span=5).mean()

    stock['ema20'] = processing_row[ma_label].ewm(span=20).mean()

    stock['ema50'] = processing_row[ma_label].ewm(span=50).mean()

    

    # # Golden Cross  implementation
    # np.where(stock['ema20'] > stock['ema50'], 1, 0)

    # stock['ema20_50_GC'] = 0 

    # for i in range(stock.shape[0]):
    #     if i == 0:
    #         pass
    #     elif stock['ema20'][i] >= stock['ema50'][i] and stock['ema20'][i-1] < stock['ema50'][i-1]: #slow code, works for now
    #         stock['ema20_50_GC'][i] = 1



    stock.dropna(inplace=True)
    

In [ ]:
# #GC value count inspction
# for stock in stock_list:
#     print(stock['ema20_50_GC'].value_counts())



In [ ]:
display(stock_list[0])

# Final data check

In [ ]:
display(stock_list[0])
display(stock_list[50])

# Models Area

In [ ]:
# stacked_lstm_1 = Sequential(

#     [

#         LSTM(units = 32, activation='tanh', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
#         LSTM(units = 32, activation='tanh', return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
#         Dense(units = 16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
#         Dense(units = 1, activation = 'linear') #output 

#     ],

#     name = "simple_stacked_lstm_stock_1"

# )

In [ ]:
# stacked_lstm_1_plus = Sequential(

#     [

#         LSTM(units = 64, activation='tanh', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
#         LSTM(units = 64, activation='tanh', return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
#         Dense(units = 24, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
#         Dense(units = 8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
#         Dense(units = 1, activation = 'linear') #output 

#     ],

#     name = "simple_stacked_lstm_stock_1_plus"

# )

# Training for all models

In [ ]:
#data storage
evaluation_result = {}


#meta data
labels = ['open']

eval_iteration = 10 #the number of evaluation iterations for individual stock

In [ ]:



#main training loop
stock_training_range = 10 #len(stock_list)

learning_rate = 0.001

for stock_index in range(50):


    #set up

    current_stock = stock_list[stock_index] #df

    column_indices = {name: i for i, name in enumerate(current_stock.columns)}

    n = len(current_stock)
    num_features = current_stock.shape[1]
    num_row = current_stock.shape[0]

    #split training, cross val and testing data
    #note since this is a time series data, the split is fixed. No randomization
    #no testing data, only cross val data

    train_data_p = 0.7  #percentage of train data
    val_data_p = 0.2

    train_df = current_stock[ 0 : int(n * train_data_p) ]
    val_df = current_stock[int( n * train_data_p) : int(n * (train_data_p + val_data_p)) ]
    test_df = current_stock[int( n * (train_data_p + val_data_p)) : ]


    #debug area
    ########################################################################
    if debug_mode and stock_index < debug_output_n :
        print("df instances : ", num_row)
        print("df features : ", num_features)
        display(current_stock)


    if debug_mode and stock_index < debug_output_n : 
        print("train_df instances : ", len(train_df))
        display(train_df)
        
        for x in range(4): 
            print()
            
        print("val_df instances : ", len(val_df))
        display(val_df)
        
        for x in range(4): 
            print()
            
        print("test_df instances : ", len(test_df))
        display(test_df)
    ########################################################################


    #normalize data
    train_mean = train_df.mean()
    train_std = train_df.std()

    #compute the z score to normalize data between features
    train_df = (train_df - train_mean)/train_std
    val_df = (val_df - train_mean)/train_std
    test_df = (test_df - train_mean)/train_std


    #visuliazation
    ########################################################################
    if debug_mode and stock_index < debug_output_n :
        df_std = (current_stock - train_mean) / train_std
        df_std = df_std.melt(var_name='Column', value_name='Normalized')
        plt.figure(figsize=(12, 6))
        ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
        _ = ax.set_xticklabels(current_stock.keys(), rotation=90)
    ########################################################################



    #window setup area
    ##############################################################################################

    #set up the 7 timesteps window
    win_7 = time_window_generator(input_width = 6, label_width = 1, shift = 1, train_df=train_df, val_df=val_df, test_df=test_df, label_columns = labels)
    #set up the batch size
    win_7.batch_size = 8

    if debug_mode and stock_index < debug_output_n :
        print(win_7)
        aux_functions.window_inspection(win_7, debug_mode, labels)
    


    #set up 30 timesteps window
    win_30 = time_window_generator(input_width = 29, label_width = 1, shift = 1, train_df=train_df, val_df=val_df, test_df=test_df, label_columns = labels)
    #set up the batch size
    win_30.batch_size = 8

    if debug_mode and stock_index < debug_output_n :
        print(win_30)
        aux_functions.window_inspection(win_30, debug_mode, labels)

    ##############################################################################################








    #individual stock model performance evaluate 
    result = []



    #train models for n numbers of iteration to check the performance
    for iter in range(eval_iteration):

        #clear backend, prevent memory overflow
        tf.keras.backend.clear_session()

        #tensorboard logging doesnt work for now
        # logpath_exact = tf_log_dir + str(stock_list[stock_index].name) + datetime.now().strftime("%Y%m%d-%H%M%S")
        # tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logpath_exact)


        #temp model area
        ##############################################################################################


        stacked_lstm_1 = Sequential(

            [

                LSTM(units = 32, activation='tanh', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
                LSTM(units = 32, activation='tanh', return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
                Dense(units = 16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
                Dense(units = 1, activation = 'linear') #output 

            ],

            name = "simple_stacked_lstm_stock_1"

        )



        stacked_lstm_2_multilr = Sequential(

            [

                LSTM(units = 64, activation='tanh', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
                LSTM(units = 64, activation='tanh', return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
                Dense(units = 24, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
                Dense(units = 8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001) ),
                Dense(units = 1, activation = 'linear') #output 

            ],

            name = "simple_stacked_lstm_2_multilr"

        )





        ##############################################################################################



        #chose model to run and evaluate
        train_target = win_30
        train_model = stacked_lstm_2_multilr



        #compile and fit
        data_model = aux_functions.compile_and_fit(train_model, train_target, epochs=60, es_patience=4, es_monitor='val_loss', es_mode='min', lr=learning_rate)


        result.append([train_model.evaluate(train_target.val), train_model.evaluate(train_target.test)])
    




    average_mape_val = 0
    average_mape_test = 0
    
    for result_i in range(len(result)):
        average_mape_val += result[result_i][0][1]
        average_mape_test += result[result_i][1][1]
    
    average_mape_val = average_mape_val/len(result)
    average_mape_test = average_mape_test/len(result)



    evaluation_result[stock_list[stock_index].name] = [average_mape_val, average_mape_test, result]



    #data logging

    if performance_log == True:

        performance_log_csv = open(data_log_csv_path, 'a', encoding='UTF8', newline='')
        csv_writer = csv.writer(performance_log_csv)
        csv_writer.writerow([stock_list[stock_index].name, average_mape_val, average_mape_test, eval_iteration, len(train_df), len(val_df), len(test_df), train_model.name])
        performance_log_csv.close()


    #garbage collection test 1
    del train_model



In [ ]:
#check all output of the dictionaries db

for key, value in evaluation_result.items():
    print("Stock Model for : ", key)
    print("Average MAPE of val data_set : ", value[0])
    print("Average MAPE of test data_set : ", value[1])
    print()

In [ ]:

# header = ["stock_code","Average MAPE of val data_set", "Average MAPE of test data_set", "Iteration_number"]   

# with open("training_log/simple_models.csv", 'w', encoding='UTF8', newline='') as file:

#     writer = csv.writer(file)

#     writer.writerow(header)

#     for key, value in evaluation_result.items():
#         writer.writerow([key, value[0], value[1]])